
## Advanced DataFrame Manipulations with PySpark: Joining, Grouping, Filtering, and Window Functions

## Task 4: Find the most popular category by country

#### Find the most popular Pinterest category people post to based on their country.
Your query should return a DataFrame that contains the following columns:
- country
- category
- category_count, a new column containing the desired query output

### Start

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa|diy-and-crafts| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria|event-planning| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| mens-fashion| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy(df_geo_alias["geo_country"], df_pin_alias["category"]).count()

# Create a window partitioned by country, ordered by the count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each country
df_top_category_per_country = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Select the desired columns
# This is the erroneours line.
####### df_most_popular = df_most_popular.select("geo_country", "category", "count")

# Filter to get only the top category (rank = 1) for each country
df_most_popular = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular = df_most_popular.withColumnRenamed("geo_country", "country")
df_most_popular = df_most_popular.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular.show()

+--------------------+--------------+--------------+
 country| category|category_count|
+--------------------+--------------+--------------+
 Afghanistan| education| 1|
 Albania| mens-fashion| 4|
 Algeria| quotes| 1|
 American Samoa| art| 1|
 Angola|diy-and-crafts| 4|
 Anguilla| home-decor| 1|
Antarctica (the t...| christmas| 1|
 Antigua and Barbuda| quotes| 1|
 Armenia|diy-and-crafts| 5|
 Aruba| travel| 1|
 Australia| mens-fashion| 1|
 Austria| travel| 1|
 Azerbaijan|event-planning| 1|
 Bahamas| art| 1|
 Bangladesh| art| 1|
 Barbados| travel| 4|
 Botswana| tattoos| 1|
Bouvet Island (Bo...| tattoos| 1|
British Virgin Is...| travel| 1|
 Bulgaria| finance| 1|
+--------------------+--------------+--------------+
only showing top 20 rows

### End

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()


DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Cambodia| 1|
 Afghanistan| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)
print(df_combined)
# Group by country and category, then count the occurrences of each category for each country
df_category_count = df_combined.groupBy("geo_country", "pin.category").count()
print(df_category_count)
# Find the maximum count for each country
max_counts = df_category_count.groupBy("geo_country").max("count")
print(max_counts)

max_counts.show()

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string, ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]
DataFrame[geo_country: string, category: string, count: bigint]
DataFrame[geo_country: string, max(count): bigint]
+--------------------+----------+
 geo_country|max(count)|
+--------------------+----------+
 Anguilla| 1|
 Fiji| 1|
 Afghanistan| 1|
 Cambodia| 1|
 Maldives| 1|
 Sudan| 1|
British Virgin Is...| 1|
 Algeria| 1|
 Angola| 4|
 Albania| 4|
 India| 1|
 Bahamas| 1|
 American Samoa| 1|
 Burundi| 1|
Central African R...| 1|
 Bangladesh| 1|
 Barbados| 4|
 Congo| 1|
Cocos (Keeling) I...| 1|
 Cote d'Ivoire| 1|
+--------------------+----------+
only showing top 20 rows

In [ ]:
from pyspark.sql.functions import col

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")
print(df_pin_alias)
print(df_geo_alias)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]
DataFrame[ind: bigint, geo_country: string, coordinates: array<double>, timestamp: timestamp]

## Task 5: Find which was the most popular category each year

Task 5: Find which was the most popular category each year

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column
df_combined = df_combined.withColumn("year", year("timestamp"))

# Group by year and category, then count the occurrences of each category for each year
df_category_count = df_combined.groupBy("year", "category").count()

# Create a window partitioned by year, ordered by the count descending
windowSpec = Window.partitionBy("year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
#df_top_category_per_year = df_top_category_per_year.withColumnRenamed("category", "most_popular_category")
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+----+--------------+--------------+
year| category|category_count|
+----+--------------+--------------+
2018| vehicles| 4|
2019|diy-and-crafts| 5|
2020| mens-fashion| 6|
2021| quotes| 2|
2022| christmas| 2|
+----+--------------+--------------+

In [ ]:
from pyspark.sql.functions import col, year
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number

# Combine the data from df_pin and df_geo
df_combined = df_pin.join(df_geo, df_pin.ind == df_geo.ind)

# Extract the year from the timestamp column and rename it to post_year
df_combined = df_combined.withColumn("post_year", year("timestamp")).drop("year")

# Group by post_year and category, then count the occurrences of each category for each post_year
df_category_count = df_combined.groupBy("post_year", "category").count()

# Create a window partitioned by post_year, ordered by the count descending
windowSpec = Window.partitionBy("post_year").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each post_year partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each post_year
df_top_category_per_year = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_category_per_year = df_top_category_per_year.withColumnRenamed("count", "category_count")

# Display the result
df_top_category_per_year.show()


+---------+--------------+--------------+
post_year| category|category_count|
+---------+--------------+--------------+
 2018| vehicles| 4|
 2019|diy-and-crafts| 5|
 2020| mens-fashion| 6|
 2021| tattoos| 2|
 2022| christmas| 2|
+---------+--------------+--------------+

## Task 6: Find the user with most followers in each country

### Step 1: For each country find the user with the most followers.
Your query should return a DataFrame that contains the following columns:

- country
- poster_name
- follower_count

In [ ]:
from pyspark.sql.functions import col, max
from pyspark.sql.window import Window

# Alias the DataFrames before joining them
df_pin_alias = df_pin.withColumnRenamed("country", "pin_country").alias("pin")
df_geo_alias = df_geo.withColumnRenamed("country", "geo_country").alias("geo")

# Join df_pin with df_geo on the index column
df_combined = df_pin_alias.join(df_geo_alias, df_pin_alias.ind == df_geo_alias.ind)

# Group by country and user (poster_name), then find the maximum follower count for each group
df_max_followers_per_user = df_combined.groupBy("geo_country", "poster_name").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.partitionBy("geo_country").orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row within each country partition
df_ranked_users = df_max_followers_per_user.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top user (rank = 1) for each country
df_top_users_per_country = df_ranked_users.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_top_users_per_country = df_top_users_per_country.withColumnRenamed("geo_country", "country")

# Select the desired columns
df_top_users_per_country = df_top_users_per_country.select("country", "poster_name", "follower_count")

# Display the result
df_top_users_per_country.show()



+--------------------+--------------------+--------------+
 country| poster_name|follower_count|
+--------------------+--------------------+--------------+
 Afghanistan| Walmart| null|
 Albania| Bored Panda| null|
 Algeria| YourTango| 942000|
 American Samoa| Thriving Home| 56000|
 Angola| CraftGossip.com| 502000|
 Anguilla|Kristen | Lifesty...| 92000|
Antarctica (the t...| HikenDip| 500000|
 Antigua and Barbuda|Sumcoco | Decor I...| 306000|
 Armenia|Michelle {CraftyM...| 892000|
 Aruba| TheTrendSpotter| 211000|
 Australia| Cultura Colectiva| null|
 Austria|Wanderlust Chloe ...| 10000|
 Azerbaijan| Style Me Pretty| null|
 Bahamas|The Kitchen Table...| 221000|
 Bangladesh| Suburban Men| 383000|
 Barbados| Nicki | 28000|
 Botswana| RapidLeaks| 4000|
Bouvet Island (Bo...| POPSUGAR| null|
British Virgin Is...|Visit USA Parks |...| 4000|
 Bulgaria|Living Low Key | ...| 26000|
+--------------------+--------------------+--------------+
only showing top 20 rows

## Step 2: Based on the above query, find the country with the user with most followers.
Your query should return a DataFrame that contains the following columns:
- country
- follower_count

This DataFrame should have only one entry.

In [ ]:
from pyspark.sql.functions import col, max

# Group by country and find the maximum follower count across all users in each country
df_max_followers_per_country = df_top_users_per_country.groupBy("country").agg(max("follower_count").alias("follower_count"))

# Create a window partitioned by country, ordered by the follower_count descending
windowSpec = Window.orderBy(col("follower_count").desc())

# Use row_number to assign a rank to each row
df_ranked_countries = df_max_followers_per_country.withColumn("rank", row_number().over(windowSpec))

# Filter to keep only the top country (rank = 1)
df_country_with_most_followers = df_ranked_countries.filter(col("rank") == 1).drop("rank")

# Display the result
df_country_with_most_followers.show()


+-------+--------------+
country|follower_count|
+-------+--------------+
Algeria| 942000|
+-------+--------------+

### Task 7: Find the most popular category for different age groups

What is the most popular category people post to based on the following age groups:
- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:
- age_group, a new column based on the original age column
- category
- category_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import when

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the combined DataFrame with df_pin on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and category, then count the occurrences of each category for each age group
df_category_count = df_combined.groupBy("age_group", "category").count()

# Create a window partitioned by age group, ordered by the count descending
windowSpec = Window.partitionBy("age_group").orderBy(col("count").desc())

# Use row_number to assign a rank to each row within each age group partition
df_ranked_categories = df_category_count.withColumn("rank", row_number().over(windowSpec))

# Filter to get only the top category (rank = 1) for each age group
df_most_popular_per_age_group = df_ranked_categories.filter(col("rank") == 1).drop("rank")

# Rename the columns
df_most_popular_per_age_group = df_most_popular_per_age_group.withColumnRenamed("count", "category_count")

# Display the result
df_most_popular_per_age_group.show()


+---------+--------------+--------------+
age_group| category|category_count|
+---------+--------------+--------------+
 18-24| mens-fashion| 17|
 25-35|diy-and-crafts| 12|
 36-50| finance| 5|
 50+| beauty| 1|
+---------+--------------+--------------+

### Task 8: Find the median follower count for different age groups

What is the median follower count for users in the following age groups:

- 18-24
- 25-35
- 36-50
- +50

Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import col, expr

# Define age groups based on the original age column
df_user_age_grouped = df_user.withColumn("age_group",
                                         when(col("age").between(18, 24), "18-24")
                                         .when(col("age").between(25, 35), "25-35")
                                         .when(col("age").between(36, 50), "36-50")
                                         .otherwise("50+"))

# Join the user DataFrame with the pin DataFrame on the index column
df_combined = df_pin.join(df_user_age_grouped, df_pin.ind == df_user_age_grouped.ind)

# Group by age group and calculate the median follower count for each age group
df_median_follower_count = df_combined.groupBy("age_group") \
    .agg(expr("percentile_approx(follower_count, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------------------+
age_group|median_follower_count|
+---------+---------------------+
 18-24| 211000|
 25-35| 43000|
 36-50| 6000|
 50+| 908|
+---------+---------------------+

## Task 9: Find how many users have joined each year?
Find how many users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- number_users_joined, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, countDistinct

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Group by the join year and count the distinct users
df_number_users_joined = df_filtered_users.groupBy("join_year").agg(countDistinct("user_name").alias("number_users_joined"))

# Display the result
df_number_users_joined.show()


+---------+-------------------+
join_year|number_users_joined|
+---------+-------------------+
 2015| 24|
 2016| 27|
 2017| 8|
+---------+-------------------+

## Task 10: Find the median follower count of users based on their joining year

Find the median follower count of users have joined between 2015 and 2020.

Your query should return a DataFrame that contains the following columns:

- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output


In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()


+-----+------------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+------------------+---+-------------------+---------+---------------------+
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 4249|AlexandriaAlvarado| 20|2015-10-23 04:13:23| 2015| 22|
 8731| AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
10138| CarolSilva| 22|2015-12-31 14:57:02| 2015| 22|
 7922| DeniseAdams| 21|2015-11-12 06:21:36| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
10883| CarlHampton| 25|2015-11-26 03:43:03| 2015| 22|
10321| AndreaBurke| 20|2015-12-12 11:20:08| 2015| 22|
+-----+------------------+---+-------------------+---------+---------------------+
only showing top 20 rows

## Task 11: Find the median follower count of users based on their joining year and age group

Find the median follower count of users that have joined between 2015 and 2020, based on which age group they are part of.
Your query should return a DataFrame that contains the following columns:

- age_group, a new column based on the original age column
- post_year, a new column that contains only the year from the timestamp column
- median_follower_count, a new column containing the desired query output

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Add a column for post_year containing only the year from the timestamp column
df_median_follower_count = df_median_follower_count.withColumn("post_year", col("join_year"))

# Reorder columns
df_median_follower_count = df_median_follower_count.select("age_group", "post_year", "median_follower_count")

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|post_year|median_follower_count|
+---------+---------+---------------------+
 20-29| 2015| 7528|
 40+| 2016| 3813|
 20-29| 2016| 5069|
 30-39| 2016| 2863|
 20-29| 2017| 5091|
 30-39| 2015| 5730|
 40+| 2017| 1611|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
#df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     #.when((col("age") >= 20) & (col("age") < 30), #"20-29")
                                                     #.when((col("age") >= 30) & (col("age") < 40), #"30-39")
                                                     #.otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()

+---------+---------------------+
join_year|median_follower_count|
+---------+---------------------+
 2015| 6145|
 2016| 4949|
 2017| 5091|
+---------+---------------------+

In [ ]:
from pyspark.sql.functions import year, expr, col
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate age group
df_filtered_users = df_filtered_users.withColumn("age_group", F.when(col("age") < 20, "0-19")
                                                     .when((col("age") >= 20) & (col("age") < 30), "20-29")
                                                     .when((col("age") >= 30) & (col("age") < 40), "30-39")
                                                     .otherwise("40+"))

# Group by age group, join year, and calculate the median follower count
df_median_follower_count = df_filtered_users.groupBy("age_group", "join_year") \
    .agg(expr("percentile_approx(ind, 0.5)").alias("median_follower_count"))

# Display the result
df_median_follower_count.show()


+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 30-39| 2016| 2863|
 20-29| 2015| 7528|
 30-39| 2015| 5730|
 20-29| 2016| 5069|
 40+| 2017| 1611|
 20-29| 2017| 5091|
 40+| 2016| 3813|
 30-39| 2017| 10625|
+---------+---------+---------------------+

In [ ]:
from pyspark.sql.functions import expr
from pyspark.sql.window import Window

# Assuming follower information is contained within df_pin, you can filter the relevant columns
df_followers = df_pin.select("ind", "follower_count")

# Now you have a DataFrame df_followers containing the unique_id and follower_count columns


# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)


df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
# Assuming df_followers is properly defined and contains the necessary columns
# Join df_filtered_users with df_followers on a common key (e.g., unique_id)
print(df_filtered_users)
print(df_followers)
print(df_followers.columns)  # This line is causing the error

# Correcting the error: Accessing the columns attribute of df_followers
print(df_followers.columns)

df_filtered_users_with_followers = df_filtered_users.join(df_followers, df_filtered_users["ind"] == df_followers["ind"], how="inner")
print(df_filtered_users)
print(df_followers)
# Once the join is successful, you can proceed with the rest of the code to calculate the median follower count

# Calculate the median follower count for each year and age group
windowSpec = Window.partitionBy("join_year", "age_group")
df_median_follower_count = df_filtered_users_with_followers.withColumn(
    "median_follower_count",
    expr("percentile_approx(follower_count, 0.5)").over(windowSpec)
).select("age_group", "join_year", "median_follower_count").distinct()

# Display the result
df_median_follower_count.show()


DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
['ind', 'follower_count']
['ind', 'follower_count']
DataFrame[ind: bigint, user_name: string, age: bigint, date_joined: timestamp, join_year: int, age_group: string]
DataFrame[ind: bigint, follower_count: int]
+---------+---------+---------------------+
age_group|join_year|median_follower_count|
+---------+---------+---------------------+
 18-24| 2015| 211000|
 25-35| 2015| 51000|
 36-50| 2015| 0|
 18-24| 2016| 502000|
 25-35| 2016| 43000|
 36-50| 2016| 17000|
 50+| 2016| 908|
 18-24| 2017| 940|
 25-35| 2017| 8000|
 36-50| 2017| 6000|
+---------+---------+---------------------+

In [ ]:
print(df_pin)

DataFrame[ind: bigint, unique_id: string, title: string, description: string, follower_count: int, poster_name: string, tag_list: string, is_image_or_video: string, image_src: string, save_location: string, category: string]

In [ ]:
from pyspark.sql.functions import year
from pyspark.sql import Window
import pyspark.sql.functions as F

# Extract the year from the date_joined column
df_user_year = df_user.withColumn("join_year", year("date_joined"))

# Filter the data for users who joined between 2015 and 2020
df_filtered_users = df_user_year.filter((col("join_year") >= 2015) & (col("join_year") <= 2020))

# Calculate the median follower count for each year
windowSpec = Window.partitionBy("join_year")
df_median_follower_count = df_filtered_users.withColumn("median_follower_count", F.expr("percentile_approx(age, 0.5)").over(windowSpec)).distinct()

# Display the result
df_median_follower_count.show()

+-----+---------------+---+-------------------+---------+---------------------+
 ind| user_name|age| date_joined|join_year|median_follower_count|
+-----+---------------+---+-------------------+---------+---------------------+
 4315| MichellePrince| 36|2015-12-20 16:38:13| 2015| 22|
 3089| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 7528| AbigailAli| 20|2015-10-24 11:23:51| 2015| 22|
 5494| AnneAllen| 27|2015-12-16 15:20:05| 2015| 22|
 9875| BrendanJoseph| 26|2015-12-20 10:28:00| 2015| 22|
 8304| CharlesBerry| 25|2015-12-28 04:21:39| 2015| 22|
 9270| AmyBrown| 21|2015-11-08 16:14:16| 2015| 22|
 4| AdamAcosta| 20|2015-10-21 21:26:45| 2015| 22|
10029| AnthonyBaker| 20|2015-11-14 19:24:56| 2015| 22|
 1667| BernardArnold| 21|2015-12-12 01:39:29| 2015| 22|
 5730| RachelDavis| 36|2015-12-08 20:02:43| 2015| 22|
 2923| BrianNelson| 26|2015-11-11 03:20:57| 2015| 22|
 2418| AmandaAdams| 20|2015-10-21 08:27:36| 2015| 22|
 857| AndrewBurke| 20|2015-11-14 17:38:31| 2015| 22|
 6063| CoreyAndrews| 23|2015-11-25 13:36:22| 2015| 22|
 6145| EmilyHawkins| 30|2015-12-25 20:24:37| 2015| 22|
 8731|AndreaAlexander| 21|2015-11-10 09:27:42| 2015| 22|
 8237| AaronAbbott| 20|2015-10-23 16:08:41| 2015| 22|
 3156| AndrewBaker| 22|2015-12-21 08:06:54| 2015| 22|
 428| ClaudiaAdams| 20|2015-11-28 02:20:29| 2015| 22|
+-----+---------------+---+-------------------+---------+---------------------+
only showing top 20 rows